## Classic Contrarian Patterns

In [1]:
import backtesting
import tradingUtils as ut
from tradingUtils import OurStrategy
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
from bokeh.io import output_notebook
output_notebook()
backtesting.set_bokeh_output(notebook=True)
import numpy as np

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BokehJS ...

In [2]:
tsla_data = ut.tsla_data

# Marabozu Pattern

Raro de ocorrer

In [3]:
def marubozu_pattern_signal(data):
    result = pd.Series(0, index=data.index)

    close = data['Adj Close'].values
    open_ = data['Open'].values
    high = data['High'].values
    low = data['Low'].values

    for i in range(len(data)):
        
        # Padrão bullish (Marubozu de alta)
        if (
            close[i] > open_[i] and 
            high[i] == close[i] and 
            low[i] == open_[i]
        ):
            result.iloc[i] = 1  # Buy signal

        # Padrão bearish (Marubozu de baixa)
        elif (
            close[i] < open_[i] and 
            high[i] == open_[i] and 
            low[i] == close[i]
        ):
            result.iloc[i] = -1  # Sell signal

    return result



In [4]:
# Gerando os sinais que dão match com a política
policy = marubozu_pattern_signal(tsla_data)
ut.exit_points(tsla_data, policy, 5, 1)

tsla_data["Signal"] = 0
tsla_data.loc[policy.index, "Signal"] = policy

tsla_data


,Open,High,Low,Close,Adj Close,Volume,Signal
Date,,,,,,,
2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500,0
2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500,0
2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000,0
2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000,0
2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500,0
...,...,...,...,...,...,...,...
2024-08-15,205.020004,215.880005,204.820007,214.139999,214.139999,89848500,0
2024-08-16,211.149994,219.800003,210.800003,216.119995,216.119995,88765100,0
2024-08-19,217.070007,222.979996,214.089996,222.720001,222.720001,76435200,0


In [5]:
# Rodando o backtest
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                    0.196574
Equity Final [$]                 10423.954779
Equity Peak [$]                  10677.634779
Return [%]                           4.239548
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                    0.294265
Volatility (Ann.) [%]                1.683178
Sharpe Ratio                         0.174827
Sortino Ratio                        0.471345
Calmar Ratio                         0.112883
Max. Drawdown [%]                   -2.606805
Avg. Drawdown [%]                   -1.977101
Max. Drawdown Duration     4388 days 00:00:00
Avg. Drawdown Duration     2196 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       3.570243
Worst Trade [%]                      0.646496
Avg. Trade [%]                    

## Three Candles


In [6]:
# Política de três candles como mesmo sinal
def three_candles_policy(data, body):
    result = pd.Series(0, index=data.index)

    # Começamos da terceira linha
    for i in range(2, len(data)):
        close = data['Adj Close'].values
        open_ = data['Open'].values

        # Checamos as condições de bullish
        if (
            (close[i] - open_[i] > body) and
            (close[i - 1] - open_[i - 1] > body) and
            (close[i - 2] - open_[i - 2] > body) and
            close[i] > close[i - 1] and
            close[i - 1] > close[i - 2]
        ):
            result.iloc[i] = 1  # Buy signal
        
        # Checamos as condições de bearish
        elif (
            (close[i] - open_[i] > body) and
            (close[i - 1] - open_[i - 1] > body) and
            (close[i - 2] - open_[i - 2] > body) and
            close[i] < close[i - 1] and
            close[i - 1] < close[i - 2]
        ):
            result.iloc[i] = -1  # Sell signal
    
    return result

In [7]:
# Gerando os sinais que dão match com a política
policy = three_candles_policy(tsla_data, 0.1)
ut.exit_points(tsla_data, policy, 5, 1)

tsla_data["Signal"] = 0
tsla_data.loc[policy.index, "Signal"] = policy

# Rodando o backtest
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                   11.541702
Equity Final [$]                 10673.917305
Equity Peak [$]                  13559.907428
Return [%]                           6.739173
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                    0.462593
Volatility (Ann.) [%]               18.437718
Sharpe Ratio                         0.025089
Sortino Ratio                        0.036727
Calmar Ratio                         0.011145
Max. Drawdown [%]                  -41.505972
Avg. Drawdown [%]                  -26.675368
Max. Drawdown Duration     3068 days 00:00:00
Avg. Drawdown Duration     1020 days 00:00:00
# Trades                                  111
Win Rate [%]                        40.540541
Best Trade [%]                      42.505807
Worst Trade [%]                    -14.029062
Avg. Trade [%]                    

## Tasuki Pattern

In [8]:
def tasuki_pattern_policy(data):
    result = pd.Series(0, index=data.index)

    # Pegamos a coluna close e open  
    close = data['Adj Close'].values
    open_ = data['Open'].values
    
    # Começamos da terceira linah
    for i in range(2, len(data)):

        # tasuki de alta
        if (
            (close[i - 2] > open_[i - 2]) and 
            (close[i - 1] > open_[i - 1]) and 
            (open_[i - 1] > close[i - 2]) and  # Checa o gap
            (open_[i] < close[i - 1]) and  # Abre no corpo do segudno
            (close[i] < open_[i]) and  # é de baixa
            (close[i] > close[i - 2])  
        ):
            result.iloc[i] = 1  

        # Tasuki de baixa
        elif (
            (close[i - 2] < open_[i - 2]) and  
            (close[i - 1] < open_[i - 1]) and  
            (open_[i - 1] < close[i - 2]) and 
            (open_[i] > close[i - 1]) and  
            (close[i] > open_[i]) and  
            (close[i] < close[i - 2]) 
        ):
            result.iloc[i] = -1  

    return result

In [9]:
# Gerando os sinais que dão match com a política
policy = tasuki_pattern_policy(tsla_data)
ut.exit_points(tsla_data, policy, 5, 1)

tsla_data["Signal"] = 0
tsla_data.loc[policy.index, "Signal"] = policy

# Rodando o backtest
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                    7.245156
Equity Final [$]                   4770.97767
Equity Peak [$]                  10433.538139
Return [%]                         -52.290223
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                   -5.102204
Volatility (Ann.) [%]               11.138212
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -60.429555
Avg. Drawdown [%]                  -36.000798
Max. Drawdown Duration     3997 days 00:00:00
Avg. Drawdown Duration     2464 days 00:00:00
# Trades                                  108
Win Rate [%]                        44.444444
Best Trade [%]                      13.803054
Worst Trade [%]                    -24.429479
Avg. Trade [%]                    

## Three Methods Pattern

Não teve nenhum sinal

## Hikkake Pattern

In [10]:
def hikkake_pattern_policy(data):
    result = pd.Series(0, index=data.index)

    close = data['Adj Close'].values
    open_ = data['Open'].values
    high = data['High'].values
    low = data['Low'].values

    for i in range(4, len(data)):  # Start from the 5th data point
        # Bullish Hikkake pattern
        if (
            close[i] > high[i - 3] and
            close[i] > close[i - 4] and
            low[i - 1] < open_[i] and
            close[i - 1] < close[i] and
            high[i - 1] <= high[i - 3] and
            low[i - 2] < open_[i] and
            close[i - 2] < close[i] and
            high[i - 2] <= high[i - 3] and
            high[i - 3] < high[i - 4] and
            low[i - 3] > low[i - 4] and
            close[i - 4] > open_[i - 4]
        ):
            result.iloc[i] = 1  # Buy signal for the next day

        # Bearish Hikkake pattern
        elif (
            close[i] < low[i - 3] and
            close[i] < close[i - 4] and
            high[i - 1] > open_[i] and
            close[i - 1] > close[i] and
            low[i - 1] >= low[i - 3] and
            high[i - 2] > open_[i] and
            close[i - 2] > close[i] and
            low[i - 2] >= low[i - 3] and
            low[i - 3] > low[i - 4] and
            high[i - 3] < high[i - 4] and
            close[i - 4] < open_[i - 4]
        ):
            result.iloc[i] = -1  # Sell signal for the next day

    return result


In [11]:
# Gerando os sinais que dão match com a política
policy = hikkake_pattern_policy(tsla_data)
ut.exit_points(tsla_data, policy, 5, 1)

tsla_data["Signal"] = 0
tsla_data.loc[policy.index, "Signal"] = policy

# Rodando o backtest
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                     1.26369
Equity Final [$]                 11942.864911
Equity Peak [$]                  12186.614911
Return [%]                          19.428649
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                    1.264381
Volatility (Ann.) [%]                4.654065
Sharpe Ratio                         0.271672
Sortino Ratio                        0.471918
Calmar Ratio                         0.146084
Max. Drawdown [%]                   -8.655173
Avg. Drawdown [%]                   -4.596725
Max. Drawdown Duration     1916 days 00:00:00
Avg. Drawdown Duration      462 days 00:00:00
# Trades                                   15
Win Rate [%]                             60.0
Best Trade [%]                       5.358688
Worst Trade [%]                      -4.42278
Avg. Trade [%]                    